In [57]:
import numpy as np
import random
import itertools

np.random.seed(0)
random.seed(0)

In [58]:
# Few helper functions
def fst(arr): return arr[0]
def snd(arr): return arr[1]
def random_bool_array(n, ratio=.5): return list(np.random.uniform(size=(n,)) > ratio)

Nejprve si vytvoříme datové typy pro jedince a zadání.
Jedince budeme kódovat jednoduše jako seznam prvků, které budou v batohu.

In [59]:
class BagItem:
	def __init__(self, price, size):
		self.price = price
		self.size = size
	
	def __repr__(self):
		return f"BagItem(price={self.price}, size={self.size})"

class Problem:
	def __init__(self, bag_size, items):
		self.bag_size = bag_size
		self.items = items
	
	@staticmethod
	def load_from_file(filename):
		with open(filename) as f:
			item_count, bag_size = list(map(int, f.readline().split()))

			items = [
				BagItem(*map(int, f.readline().split())) for _ in range(item_count)
			]

			return Problem(bag_size, items)

In [128]:
class Individual:
	def __init__(self, selected_items, problem):
		self.selected_items = selected_items
		self.problem = problem
	
	@staticmethod
	def random_for(problem):
		return Individual(random_bool_array(len(problem.items)), problem)
	
	@staticmethod
	def empty_for(problem):
		return Individual([False for _ in problem.items], problem)
	
	def get_selected_items(self):
		return map(snd, filter(fst,
			zip(self.selected_items, self.problem.items)
		))
	
	def __str__(self):
		return f"Individual({', '.join(map(str, self.get_selected_items()))})"

Nyní musíme vytvořit operace na jedincích. Například fitness funkci nebo mutace.

Fitness funkce bude jednoduše součet hodnot prvků v batohu. Pokud ovšem bude součet objemů větší než kolik se vejde do batohu, tak vrátíme něco záporného (čím víc přesahujeme velikostí, tím menší číslo).
To totiž přesně vystihuje to co od nejlepšího řešení čekáme a zároveň nám dává návod, jak se dostat z řešení, které přesahuje objemem do validního řešení.

Mutace budeme mít náhodné spojení (item-wise), případně budeme mít mutaci na jedinci, která změní náhodné prvky.

In [155]:
def fitness(individual):
	bag_size = individual.problem.bag_size
	
	selected_items_size = 0
	selected_items_price = 0

	for selected_item in individual.get_selected_items():
		selected_items_price += selected_item.price
		selected_items_size += selected_item.size
	
	if selected_items_size > bag_size:
		# This solution is not acceptable but we need to hint, on how to fix it
		return bag_size - selected_items_size
	
	return selected_items_price

def select_survivors(individuals, n):
	fitnesses = zip(
		map(fitness, individuals),
		individuals
	)

	fitnesses = map(snd,
		reversed(sorted(fitnesses, key=fst))
	)

	return list(fitnesses)[:n]

def mix_individuals(a, b, ratio=.5):
	a_bitmap = a.selected_items
	b_bitmap = b.selected_items

	picked_version = random_bool_array(len(a_bitmap), ratio)

	final_bitmap = [
		a_bitmap[i] if picked_version[i] else b_bitmap[i]
		for i in range(len(picked_version))
	]

	return Individual(final_bitmap, a.problem)

def mutate_single(individual, p):
	return mix_individuals(individual, Individual.random_for(individual.problem), p)


Pár testů našich funkcí.

In [158]:
problem = Problem(10, [
	BagItem(10, 2),
	BagItem(5, 1),
	BagItem(8, 3),
	BagItem(1, 10),
])
a = Individual.random_for(problem)
b = Individual.random_for(problem)
print("Generated a",  a)
print("Generated b",  b)
print()

mix_ab = mix_individuals(a, b)
print("Mixed", mix_ab)
print()

print("Single mutation a:", mutate_single(a, .1))



Generated a Individual(BagItem(price=1, size=10))
Generated b Individual(BagItem(price=10, size=2), BagItem(price=5, size=1), BagItem(price=1, size=10))

Mixed Individual(BagItem(price=5, size=1), BagItem(price=1, size=10))

Single mutation a: Individual(BagItem(price=1, size=10))


Teď můžeme vytvořit učící algoritmus.

Ten prostě pojede po generacích a v každé si vezme n nejlepších jedinců a z nich vytvoří další generaci.
Budeme chtít zachovávat nejlepší jedince (ať nepřijdeme o nějaké řešení co jsme vymysleli).
Zbytek generace vygenerujeme náhodných spojováním nejlepších jedinců.

In [159]:
def next_generation(population, n, generation_size, mutation_p):
	survivors = select_survivors(population, n)
	possible_parents = list(itertools.combinations(survivors, 2))

	# Keep best individuals in next generation
	children = survivors
	
	# We also want mutated survivors
	children += list(map(lambda x: mutate_single(x, mutation_p), survivors))

	# Generate other children from possible parents
	while len(children) < generation_size:
		children.append(
			mutate_single(
				mix_individuals(
					*random.choice(possible_parents)
				),
				mutation_p
			)
		)
	return children

In [160]:
def compute_problem(file_path, generation_size=100, generations=100, surviving_percent=.2, mutation_percent=.1):
	problem = Problem.load_from_file(file_path)

	population = []


	# Let's add few empty individuals (converges faster with smaller bag sizes)
	while len(population) < generation_size / 2:
		population.append(Individual.empty_for(problem))

	# Add random individuals
	while len(population) < generation_size:
		population.append(Individual.random_for(problem))

	for generation_idx in range(generations):
		population = list(next_generation(population, int(generation_size * surviving_percent), generation_size, mutation_percent))
		print("Generation", generation_idx, "best fitness:", max(map(fitness, population)))
	
	return max(population, key=fitness)

In [165]:
A = 'domaci_ukol_data/debugging_data_10.txt'
B = 'domaci_ukol_data/debugging_data_20.txt'
C = 'domaci_ukol_data/input_data_100.txt'
D = 'domaci_ukol_data/input_data_1000.txt'

In [166]:

print(compute_problem(A, generation_size=100, generations=400, surviving_percent=.3, mutation_percent=.01))

Generation 0 best fitness: 280
Generation 1 best fitness: 280
Generation 2 best fitness: 290
Generation 3 best fitness: 290
Generation 4 best fitness: 294
Generation 5 best fitness: 294
Generation 6 best fitness: 295
Generation 7 best fitness: 295
Generation 8 best fitness: 295
Generation 9 best fitness: 295
Generation 10 best fitness: 295
Generation 11 best fitness: 295
Generation 12 best fitness: 295
Generation 13 best fitness: 295
Generation 14 best fitness: 295
Generation 15 best fitness: 295
Generation 16 best fitness: 295
Generation 17 best fitness: 295
Generation 18 best fitness: 295
Generation 19 best fitness: 295
Generation 20 best fitness: 295
Generation 21 best fitness: 295
Generation 22 best fitness: 295
Generation 23 best fitness: 295
Generation 24 best fitness: 295
Generation 25 best fitness: 295
Generation 26 best fitness: 295
Generation 27 best fitness: 295
Generation 28 best fitness: 295
Generation 29 best fitness: 295
Generation 30 best fitness: 295
Generation 31 best

In [167]:
print(compute_problem(B, generation_size=100, generations=400, surviving_percent=.3, mutation_percent=.01))

Generation 0 best fitness: 854
Generation 1 best fitness: 915
Generation 2 best fitness: 920
Generation 3 best fitness: 923
Generation 4 best fitness: 958
Generation 5 best fitness: 958
Generation 6 best fitness: 966
Generation 7 best fitness: 997
Generation 8 best fitness: 1024
Generation 9 best fitness: 1024
Generation 10 best fitness: 1024
Generation 11 best fitness: 1024
Generation 12 best fitness: 1024
Generation 13 best fitness: 1024
Generation 14 best fitness: 1024
Generation 15 best fitness: 1024
Generation 16 best fitness: 1024
Generation 17 best fitness: 1024
Generation 18 best fitness: 1024
Generation 19 best fitness: 1024
Generation 20 best fitness: 1024
Generation 21 best fitness: 1024
Generation 22 best fitness: 1024
Generation 23 best fitness: 1024
Generation 24 best fitness: 1024
Generation 25 best fitness: 1024
Generation 26 best fitness: 1024
Generation 27 best fitness: 1024
Generation 28 best fitness: 1024
Generation 29 best fitness: 1024
Generation 30 best fitness: 

In [170]:
print(compute_problem(C, generation_size=100, generations=400, surviving_percent=.3, mutation_percent=.01))

Generation 0 best fitness: 1277
Generation 1 best fitness: 1277
Generation 2 best fitness: 2208
Generation 3 best fitness: 2552
Generation 4 best fitness: 2945
Generation 5 best fitness: 3203
Generation 6 best fitness: 3483
Generation 7 best fitness: 3746
Generation 8 best fitness: 4387
Generation 9 best fitness: 4564
Generation 10 best fitness: 4564
Generation 11 best fitness: 4864
Generation 12 best fitness: 4915
Generation 13 best fitness: 5419
Generation 14 best fitness: 5505
Generation 15 best fitness: 5979
Generation 16 best fitness: 6059
Generation 17 best fitness: 6059
Generation 18 best fitness: 6059
Generation 19 best fitness: 6059
Generation 20 best fitness: 6516
Generation 21 best fitness: 6516
Generation 22 best fitness: 6757
Generation 23 best fitness: 6757
Generation 24 best fitness: 6757
Generation 25 best fitness: 6757
Generation 26 best fitness: 6757
Generation 27 best fitness: 6757
Generation 28 best fitness: 6757
Generation 29 best fitness: 6757
Generation 30 best f

In [172]:
print(compute_problem(D, generation_size=50, generations=1000, surviving_percent=.1, mutation_percent=.01))

Generation 0 best fitness: 6448
Generation 1 best fitness: 10072
Generation 2 best fitness: 10072
Generation 3 best fitness: 11113
Generation 4 best fitness: 11113
Generation 5 best fitness: 11134
Generation 6 best fitness: 11584
Generation 7 best fitness: 13347
Generation 8 best fitness: 13527
Generation 9 best fitness: 14221
Generation 10 best fitness: 16186
Generation 11 best fitness: 16186
Generation 12 best fitness: 16186
Generation 13 best fitness: 16186
Generation 14 best fitness: 16671
Generation 15 best fitness: 17649
Generation 16 best fitness: 18243
Generation 17 best fitness: 18243
Generation 18 best fitness: 18243
Generation 19 best fitness: 19562
Generation 20 best fitness: 19562
Generation 21 best fitness: 20032
Generation 22 best fitness: 20032
Generation 23 best fitness: 20032
Generation 24 best fitness: 21347
Generation 25 best fitness: 21347
Generation 26 best fitness: 21347
Generation 27 best fitness: 21347
Generation 28 best fitness: 21962
Generation 29 best fitnes

Nás evoluční algoritmus nenachází optimální řešení ale většinou najde rozumně rychle něco dostatečně dobrého (pro debugovací vstupy najde optimální řešení).